## Chatbot com resumo de mensagens

LLMs para gerar um resumo contínuo da conversa.

Isso nos permite manter uma representação compacta da conversa completa, em vez de simplesmente removê-la com cortes ou filtros.

Incorporaremos esse resumo em um chatbot simples.

E equiparemos esse chatbot com memória, permitindo conversas de longa duração sem incorrer em alto custo de tokens/latência.

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o",temperature=0)

In [ ]:
from langgraph.graph import MessagesState
class State(MessagesState):
    summary: str

### O que é state.get("summary", "")?

* O método .get() tenta recuperar o valor associado à chave "summary" dentro do objeto state.

* Se a chave "summary" existir, seu valor é atribuído à variável summary.

* Se a chave "summary" não existir (ou for None), o método .get() retorna o valor padrão fornecido como segundo argumento, que é uma string vazia ("").

* Propósito: Garante que a variável summary sempre terá um valor (ou o resumo existente, ou uma string vazia), evitando erros se o campo de resumo não tiver sido preenchido no estado.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

def call_model(state: State):

    summary = state.get("summary", "")

    if summary:

        system_message = f"Summary of conversation earlier: {summary}"

        messages= [SystemMessage(content= system_message)] + state["messages"]

    else:
        messages= state["messages"]

    response= model.invoke(messages)
    return {"messages": response}

In [ ]:
def summarize_conversation(state: State):
    
    summary = state.get("summary", "")

    # Crie nosso prompt de resumo
    if summary:
        
        # Já existe um resumo
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    # Adicionar prompt ao nosso histórico
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

In [ ]:
from langgraph.graph import END
from typing_extensions import Literal
# Determine whether to end or summarize the conversation
def should_continue(state: State) -> Literal ["summarize_conversation", END]:
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

## Adicionando memória
Lembre-se de que o estado é transitório para uma única execução do grafo.

Isso limita nossa capacidade de ter conversas com múltiplas etapas e interrupções.

Como apresentado no final do Módulo 1, podemos usar persistência para resolver isso!

O LangGraph pode usar um checkpoint para salvar automaticamente o estado do grafo após cada etapa.

Essa camada de persistência integrada fornece memória, permitindo que o LangGraph retome a execução a partir da última atualização de estado.

Como mostramos anteriormente, uma das opções mais fáceis de usar é o MemorySaver, um armazenamento de chave-valor em memória para o estado do grafo.

Tudo o que precisamos fazer é compilar o grafo com um checkpoint e nosso grafo terá memória!

In [ ]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

## Threads

O mecanismo de checkpoint salva o estado a cada passo como um ponto de verificação.

Esses pontos de verificação salvos podem ser agrupados em uma sequência de conversa.

Pense no Slack como uma analogia: diferentes canais carregam diferentes conversas.

As sequências são como canais do Slack, capturando conjuntos agrupados de estado (por exemplo, conversas).

Abaixo, usamos um recurso configurável para definir um ID de sequência.

In [ ]:
## Criando uma thread
config= {"configurable": {"thread_id": "1"}}

# Iniciando a conversa
input_message= HumanMessage(content= "Hi! I'm Lance")
output= graph.invoke({"messages": [input_message]}, config)
for m in output["messages"][-1:]:
    m.pretty_print()

input_message= HumanMessage(content= "what's my name?")
output= graph.invoke({"messages": [input_message]}, config)
for m in output["messages"][-1:]:
    m.pretty_print()

input_message= HumanMessage(content= "I like the 49ers!")
output= graph.invoke({"messages": [input_message]}, config)
for m in output["messages"][-1:]:
    m.pretty_print()


Como definimos para possuir um resumo somente após possuir mais de 6 mensagens, nós ainda não temos um resumo

In [ ]:
graph.get_state(config).values.get("summary", "")

In [ ]:
input_message = HumanMessage(content="i like Nick Bosa, isn't he the highest paid defensive player?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

In [ ]:
graph.get_state(config).values.get("summary","")